In [1]:
# import findspark and os and let findspark find all the environment variables
import findspark
import os
findspark.init()


# Before you create the SparkSession, you need to add a new environment variable 
# to tell pyspark where the graphframes library is
SUBMIT_ARGS = "--packages graphframes:graphframes:0.7.0-spark2.4-s_2.11 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import udf

# Create the SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("graphx-lab").getOrCreate()

# import additional libraries
from graphframes import *
import hashlib

In [2]:
sc = spark.sparkContext 
sc.setCheckpointDir('.')

In [9]:
df = spark.read.load(["test/part-00000","test/part-00001"],
                     format="csv", sep="\t", inferSchema="true", header="false")

In [41]:
path_list('test', [2,4,6,8])

[['test/part-00000', 'test/part-00001'],
 ['test/part-00000', 'test/part-00001', 'test/part-00002', 'test/part-00003'],
 ['test/part-00000',
  'test/part-00001',
  'test/part-00002',
  'test/part-00003',
  'test/part-00004',
  'test/part-00005'],
 ['test/part-00000',
  'test/part-00001',
  'test/part-00002',
  'test/part-00003',
  'test/part-00004',
  'test/part-00005',
  'test/part-00006',
  'test/part-00007']]

In [29]:
def hashnode(x):
    return hashlib.sha1(x.encode("UTF-8")).hexdigest()[:8]

hashnode_udf = udf(hashnode)

def path_list(path, size):
    ps = []
    for s in size:
        ps.append([path+'/part-'+str(i).zfill(5) for i in range(s)])
#     print(ps)
    return ps

def build_graph(df):
    df = df.withColumnRenamed('_c0', 'parentTLD').withColumnRenamed('_c1', 'childTLD').filter("parentTLD is not null and childTLD is not null")
    df = df.drop_duplicates()
    aggcodes = df.select("parentTLD","childTLD").rdd.flatMap(lambda x: x).distinct()
    vertices = aggcodes.map(lambda x: (hashnode(x), x)).toDF(["id","name"])
    edges = df.select("parentTLD","childTLD")\
    .withColumn("src", hashnode_udf("parentTLD"))\
    .withColumn("dst", hashnode_udf("childTLD")).select("src","dst")
#     print(edges.count())
#     print(vertices.count())
    return GraphFrame(vertices, edges), vertices, edges


def density(path='test', size=[1,2,4,8]):
    dens = []
    for l in path_list(path, size):
        df = spark.read.load(l,format="csv", sep="\t", inferSchema="true", header="false")
        graph,vertices,edges  = build_graph(df)
        
        n_v = vertices.count()
        n_e = edges.count()
        
        dens.append(n_e / (n_v*(n_v-1)))
    return dens

In [43]:
density('test', [1,2])

[1.6005258989577336e-05, 9.490307338509154e-06]

In [39]:
# graph,vertices,edges  = build_graph(df)
# result = graph.labelPropagation(maxIter=5)
# result.cache()
result.show(5)
# result.groupby('label').count().orderBy("count", ascending=False).show()
print(result.dropDuplicates(['label']).count())
vertices.distinct().count()

+--------+--------------------+-------------+
|      id|                name|        label|
+--------+--------------------+-------------+
|0738ebbe|   moritzlaw.osu.edu|1142461301062|
|09976f76|cookingonabootstr...|1357209666145|
|9d3f3877|d301l49b0xhsa2.cl...| 747324310219|
|179fbf80|    logi103.xiti.com|  17179869532|
|766cca87|  statistics.bczp.cn|1262720385652|
+--------+--------------------+-------------+
only showing top 5 rows



DataFrame[id: string, name: string]

In [ ]:
def indegree(dataframe, website=['twitter.com'], sizes=[1,2,3]):
    '''input: dataframe, list: websites tlds; list: number of files
       output: dataframe
+------------+--------+---------------------------+
|website     |size_1  |sizs_2    |sizse_3             |
+------------+--------+---------------------------+
|twitter.com |1       | 100      |
|05f4dd35    |1       |          |
|033e4ad5    |1       |          |
|00615816    |1       |          |
|034274db    |1       |          |
+-------------+--------+---------------------------+
    '''
    return None

In [ ]:
indegree([2,4,8,16,32,64,128,256,512,1024])

In [ ]:
indegree_df = indegree(df, ['twitter.com', 'baidu,com', 'google'], [2,4,8,16])
indegree_df.show()

In [ ]:
def outdegree(dataframe, website=['twitter.com'], sizes=[1,2,3]):
    '''input: dataframe, list: websites tlds; list: number of files
       output: dataframe
+------------+--------+---------------------------+
|website     |size_1  |sizs_2    |  sizse_3             |
+------------+--------+---------------------------+
|twitter.com |1       | 100      |
|05f4dd35    |1       |          |
|033e4ad5    |1       |          |
|00615816    |1       |          |
|034274db    |1       |          |
+------------+--------+---------------------------+
    '''
    return None

In [ ]:
def connected_components(dataframe, sizes=[1,2,3]):
    '''input: dataframe, list: number of files
       output: dataframe
+------------+--------+---------------------------+
|size        |number of componemts  |sizs_2    |  sizse_3             |
+------------+--------+---------------------------+
|1           |1                     | 100      |
|2           |1                     |          |
|3           |1                     |          |

    '''
    return None

In [ ]:
def analyze_graph(path='test', size=[2,4,8,16]):
    d = {}
    
    d['indeg_df']  = indegree(size=size)
    d['ourdeg_df'] = outdegree(size=size)
    d['cc'] = connected_components(size=size)
    
    return d

In [6]:
df.cache()

DataFrame[_c0: string, _c1: string]

In [4]:
df.show(10)

+---------+-----------------+
|      _c0|              _c1|
+---------+-----------------+
|002104.cn|    hd.hstong.com|
|002104.cn|   stock.stcn.com|
|002104.cn|  upload.stcn.com|
|002104.cn|  upload.stcn.com|
|002104.cn|  upload.stcn.com|
|002104.cn|  upload.stcn.com|
|002446.cn|    hd.hstong.com|
|003145.cn|qile.qiquting.com|
|003145.cn|qile.qiquting.com|
|003991.cn|qile.qiquting.com|
+---------+-----------------+
only showing top 10 rows



In [8]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [7]:
df.count()

3678351

In [9]:

df.show(5)

+---------------+------------------+
|      parentTLD|          childTLD|
+---------------+------------------+
|24hourebook.com|       jlsl.gov.cn|
|   350.xn--p1ai|   connect.mail.ru|
|   350.xn--p1ai|zakladki.yandex.ru|
|        363v.cn|      pk105555.com|
|      87nara.kr|       iampine.com|
+---------------+------------------+
only showing top 5 rows



In [16]:
df.count()

904373

In [13]:

print(aggcodes.count())
aggcodes.top(10)

['바로잡기.com',
 '무지안내.net',
 '맥가이버피아노.com',
 '라포끌레르.kr',
 '더풋샵.net',
 '나로.한국',
 '강원곳간.com',
 '非常靠谱.com',
 '陕西文化网.中国',
 '陕西文化网.com']

+--------+--------------------+
|      id|                name|
+--------+--------------------+
|64810973|img514.imageshack.us|
|465806fb|         twitter.com|
|247e1a71|  beautifulshoes.org|
|5584fcd3|guccihandbagsoutl...|
|09651082|       bufengcong.cn|
+--------+--------------------+
only showing top 5 rows



In [18]:


edges.show(5)

+--------+--------+
|     src|     dst|
+--------+--------+
|a955fc5f|ec156376|
|2b778455|b1045852|
|2b778455|a49aa74f|
|e8b6308e|ebd38990|
|b0451246|3fcdbfdc|
+--------+--------+
only showing top 5 rows



In [20]:
graph.vertices.show(5)
graph.edges.show(5)

+--------+--------------------+
|      id|                name|
+--------+--------------------+
|64810973|img514.imageshack.us|
|465806fb|         twitter.com|
|247e1a71|  beautifulshoes.org|
|5584fcd3|guccihandbagsoutl...|
|09651082|       bufengcong.cn|
+--------+--------------------+
only showing top 5 rows

+--------+--------+
|     src|     dst|
+--------+--------+
|a955fc5f|ec156376|
|2b778455|b1045852|
|2b778455|a49aa74f|
|e8b6308e|ebd38990|
|b0451246|3fcdbfdc|
+--------+--------+
only showing top 5 rows



In [22]:
graph.inDegrees.join(vertices, on="id")\
.orderBy("inDegree", ascending=False).show(20, False)
graph.outDegrees.join(vertices, on="id")\
.orderBy("outDegree", ascending=False).show(20, False)

+--------+--------+---------------------------+
|id      |inDegree|name                       |
+--------+--------+---------------------------+
|03111be4|1       |bigjeans.co.kr             |
|05f4dd35|1       |kitp.ucsb.edu              |
|033e4ad5|1       |livefutbol.com             |
|00615816|1       |weekendhotel.blog          |
|034274db|1       |vusurscene.com             |
|0088f064|1       |topfreizeit.de             |
|035a0f78|1       |donemd.cn                  |
|00ade0f6|1       |printshop.co.kr            |
|05d51afe|1       |ehiln67n99.353qt.cn        |
|0380adbc|1       |kaatru.in                  |
|00bf6f09|1       |hg8868fdzs71.cn.com        |
|03ac6afd|1       |nmyk.kingdns.in            |
|0133f1a7|1       |qualitywellnessresorts.nl  |
|03fdc338|1       |sarquest.ru                |
|01804c23|1       |dosenhor.com               |
|0404cf3e|1       |167.114.119.122:9676       |
|019a4650|1       |silkroomthailand.com       |
|04128bf0|1       |openparachute.wordpre

In [35]:
# result = graph.connectedComponents()
result.select("id", "component").groupby('component').count().orderBy("count", ascending=False).show()

+-----------+------+
|  component| count|
+-----------+------+
|          0|459137|
|        116|  3499|
|       1064|   982|
|         75|   622|
|       1387|   519|
|        732|   451|
|        339|   335|
|       2349|   308|
|       1238|   277|
|        184|   244|
|        516|   241|
| 8589936039|   239|
|        856|   228|
|       2061|   226|
|       2398|   226|
|       1301|   180|
| 8589934891|   178|
|       2506|   172|
|       1666|   168|
|34359739686|   135|
+-----------+------+
only showing top 20 rows



In [36]:
result2 = graph.stronglyConnectedComponents(maxIter=10)
result2.select("id", "component").orderBy("component").show()

+--------+---------+
|      id|component|
+--------+---------+
|002c9569|        0|
|008c2d9c|        1|
|00927e3f|        2|
|00947a5b|        3|
|00acfdcd|        4|
|00ad5293|        5|
|00c6dae1|        6|
|00d7cab4|        7|
|010b6f81|        8|
|010ba2b9|        9|
|014f3a50|       10|
|01703f81|       11|
|0191598e|       12|
|01bc71ce|       13|
|01c087cc|       14|
|01d4353c|       15|
|01dfa2cd|       16|
|0227d662|       17|
|02341045|       18|
|0241ee58|       19|
+--------+---------+
only showing top 20 rows



In [ ]:
result3 = g.labelPropagation(maxIter=5)
result4.select("id", "label").show()

In [ ]:
results = g.pageRank(resetProbability=0.15, tol=0.01)
results.vertices.select("id", "pagerank").show()
results.edges.select("src", "dst", "weight").show()

In [1]:
spark.stop()

NameError: name 'spark' is not defined